## <u>docker commands</u>

docker pull cassandra:latest

docker run --name cassandra-container -p 9042:9042 -d cassandra:latest

docker exec -it cassandra-container cqlsh


In [ ]:
import numpy as np
import pandas as pd
import csv
from cassandra.cluster import Cluster
from cassandra.query import BatchStatement, ConsistencyLevel, BatchType
import os


## <u>setting up dataframes <u>

In [ ]:
base_path = "data"

artist_df = pd.read_csv(os.path.join(base_path,'spotify_artist_data_2023.csv'))
albums_df = pd.read_csv(os.path.join(base_path,'spotify-albums_data_2023.csv'))
features_df = pd.read_csv(os.path.join(base_path,'spotify_features_data_2023.csv'))
tracks_df = pd.read_csv(os.path.join(base_path,'spotify_tracks_data_2023.csv'))
data_df = pd.read_csv(os.path.join(base_path,'spotify_data_12_20_2023.csv'))

In [ ]:
##setting up our albums df

drop_columns = ['artist_0', 'artist_1', 'artist_2', 'artist_3', 'artist_4', 'artist_5', 'artist_6', 'artist_7', 'artist_8', 'artist_9', 'artist_10', 'artist_11']

# Fill 'artists' column containing lists of non-null artists
albums_df['artists'] = albums_df.apply(lambda row: [value for key, value in row.items() if key in drop_columns and pd.notna(value)], axis=1)

albums_df = albums_df.drop(drop_columns, axis=1)

# Identify albums with missing values in the specified columns
albums_with_missing_tracks = albums_df[albums_df[albums_df.columns].isna().any(axis=1)]['album_id'].unique()

# Remove entire albums with missing values
albums_df = albums_df[~albums_df['album_id'].isin(albums_with_missing_tracks)]

albums_df = pd.merge(albums_df, tracks_df[['id', 'track_popularity']], left_on='track_id', right_on='id', how='inner')

albums_df = albums_df.drop(['id'], axis=1)

# Convert release_date to datetime
albums_df['release_date'] = pd.to_datetime(albums_df['release_date'], utc=True)

# Create release_year, release_month and release_day columns
albums_df['release_year'] = albums_df['release_date'].dt.year
albums_df['release_month'] = albums_df['release_date'].dt.month
albums_df['release_day'] = albums_df['release_date'].dt.day

# Add tracks features
features_columns = ['danceability', 'loudness', 'energy', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'id']

albums_df = pd.merge(albums_df, features_df[features_columns], left_on='track_id', right_on='id', how='inner')
albums_df = albums_df.drop(['id'], axis=1)

albums_df = pd.merge(albums_df, artist_df[['id', 'artist_genres']], left_on='artist_id', right_on='id', how='left')
albums_df = albums_df.drop(['id'], axis=1)

print(albums_df.shape[1])



## <u>code for inserting data into cassandra table </u>

note that when using batch we should make our batch all with the same partition key

below is code to enter the track info csv.  We first create seperate dataframes based on potential values for primary keys and then insert individually
    
We still need to change the code to work for our exact desired tables
    
    

In [ ]:
## connect to cassandra
try:
    cluster = Cluster(['127.0.0.1'])
    session = cluster.connect()
except Exception as e:
    print(e)

In [ ]:
## create keyspace

try:
    session.execute("""
        CREATE KEYSPACE IF NOT EXISTS spotify
        WITH REPLICATION =
        {'class': 'SimpleStrategy', 'replication_factor': 1}
    """)

except Exception as e:
    print(e)


session.set_keyspace('spotify')

In [ ]:
# session.execute("DROP TABLE spotify.track_info_release_year;")

In [ ]:
## create table 1

try:
    session.execute("""
        CREATE TABLE IF NOT EXISTS track_info_release_year (
            track_name TEXT,
            track_id TEXT,
            track_number TEXT,
            duration_ms TEXT,
            album_type TEXT,
            artists TEXT,
            total_tracks TEXT,
            album_name TEXT,
            release_date TEXT,
            label TEXT,
            album_popularity TEXT,
            album_id TEXT,
            artist_id TEXT,
            duration_sec TEXT,
            release_year TEXT,
            track_popularity TEXT,
            release_month TEXT,
            release_day TEXT,
            danceability FLOAT,
            loudness FLOAT,
            energy FLOAT,
            speechiness FLOAT,
            acousticness FLOAT,
            instrumentalness FLOAT,
            liveness FLOAT,
            valence FLOAT,
            tempo FLOAT,
            artist_genres TEXT,
            PRIMARY KEY (release_year, track_id));
    """)
except Exception as e:
    print(e)

In [ ]:
## create table 2

try:
    session.execute("""
        CREATE TABLE IF NOT EXISTS track_info_popularity (
            track_name TEXT,
            track_id TEXT,
            track_number TEXT,
            duration_ms TEXT,
            album_type TEXT,
            artists TEXT,
            total_tracks TEXT,
            album_name TEXT,
            release_date TEXT,
            label TEXT,
            album_popularity TEXT,
            album_id TEXT,
            artist_id TEXT,
            duration_sec TEXT,
            track_popularity TEXT,
            release_year TEXT,
            release_month TEXT,
            release_day TEXT,
            danceability FLOAT,
            loudness FLOAT,
            energy FLOAT,
            speechiness FLOAT,
            acousticness FLOAT,
            instrumentalness FLOAT,
            liveness FLOAT,
            valence FLOAT,
            tempo FLOAT,
            artist_genres TEXT,
            PRIMARY KEY (track_popularity, track_id));
    """)
except Exception as e:
    print(e)

In [ ]:
## reduce csv size

def limit_csv_lines(input_path, output_path, limit):
    with open(input_path, 'r', newline='') as input_file:
        csv_reader = csv.reader(input_file)
        header = next(csv_reader, None)  # Assuming the file has a header

        lines_to_write = []
        for _ in range(limit):
            try:
                row = next(csv_reader)
                lines_to_write.append(row)
            except StopIteration:
                break

    with open(output_path, 'w', newline='') as output_file:
        csv_writer = csv.writer(output_file)
        if header:
            csv_writer.writerow(header)
        csv_writer.writerows(lines_to_write)

In [ ]:
## using batch

def convert_to_types(type_list, string_list):
    converted_tuple = tuple(data_type(value) for data_type, value in zip(type_list, string_list))
    return converted_tuple

# should recieve a df with all records having the same partition key
def df_to_cassandra(df, prepared_query, types, batch_size=20, has_header=True):
    for chunk_start in range(0, len(df), batch_size):
        chunk_end = min(chunk_start + batch_size, len(df))
        chunk = df.iloc[chunk_start:chunk_end]

        batch = BatchStatement(consistency_level=ConsistencyLevel.QUORUM, batch_type=BatchType.UNLOGGED)
        for index, row in chunk.iterrows():
            converted_vals = convert_to_types(types, row.tolist())
            batch.add(prepared_query, converted_vals)
        session.execute(batch)


In [ ]:
#prep datafram

df_sorted_release_year = albums_df.sort_values(by='release_year', ascending=False)
df_sorted_popularity = albums_df.sort_values(by='track_popularity', ascending=False)

print(df_sorted_release_year.shape)

In [ ]:
df_sorted_popularity.iloc[0]

In [ ]:
#track_name, track_id,track_number ,duration_ms,album_type, artists,total_tracks ,album_name, release_date,label, album_popularity, album_id, artist_id, duration_sec, track_popularity, release_year, release_month, release_day, danceability, loudness, energy, speechiness, acousticness, instrumentalness, liveness, valence       ,tempo,

In [ ]:
# insert release year datafram to casandra table

types = [str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,float,float,float,float,float,float,float,float,float,str]

statement_track_info = '''
INSERT INTO track_info_release_year
(track_name, track_id,track_number ,duration_ms,album_type, artists,total_tracks ,album_name, release_date,label, album_popularity, album_id, artist_id, duration_sec, track_popularity, release_year, release_month, release_day, danceability, loudness, energy, speechiness, acousticness, instrumentalness, liveness, valence,tempo, artist_genres)
 VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?,?);
'''
prepared_query = session.prepare(statement_track_info)

# for max performance we should split out csv by partition
df_to_cassandra(df_sorted_release_year, prepared_query, types)


In [ ]:
# insert popularity datafram to casandra table
statement_track_info = '''
INSERT INTO track_info_popularity
(track_name,track_id,track_number,duration_ms,album_type,artists,total_tracks,album_name,release_date,label,album_popularity,album_id,artist_id,duration_sec,track_popularity, release_year,release_month,release_day,danceability,loudness,energy,speechiness,acousticness,instrumentalness,liveness,valence,tempo,artist_genres)
 VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?);
'''
prepared_query = session.prepare(statement_track_info)

# for max performance we should split out csv by partition
df_to_cassandra(df_sorted_popularity, prepared_query, types)


In [ ]:
## some queries for table partitioned by year

def find_average_of_property_from_year(property, start, end):
    avgs = []
    for year in range(start, end + 1):
        statement = f"SELECT AVG({property}) FROM track_info_release_year WHERE release_year = '{year}';"
        avgs.append(session.execute(statement).one()[0])
    return avgs

def get_all_tracks_in_year(year):
    statement = f"SELECT * FROM track_info_release_year WHERE release_year = '{year}';"
    return session.execute(statement)._current_rows




In [ ]:
## some queries for table partitioned by popularity

def find_average_of_property_with_popularity_range(property, start, end):
    avgs = []
    for popularity in range(start, end + 1):
        statement = f"SELECT AVG({property}) FROM track_info_popularity WHERE track_popularity = '{popularity}';"
        avgs.append(session.execute(statement).one()[0])
    return(avgs)

def get_all_tracks_with_popularity(popularity, column):
    statement = f"SELECT {column} FROM track_info_popularity WHERE track_popularity = '{popularity}';"
    return session.execute(statement)._current_rows


## <u>Executing queries<u>

In [ ]:
statement_count_rows_1 = "SELECT count(*) from track_info_release_year"
print(session.execute(statement_count_rows_1)._current_rows)

statement_count_rows_2 = "SELECT count(*) from track_info_popularity"
print(session.execute(statement_count_rows_2)._current_rows)

In [ ]:
find_average_of_property_from_year("tempo", 2000,2005)

In [ ]:
find_average_of_property_with_popularity_range("tempo", 90,99)

In [ ]:
get_all_tracks_with_popularity("99", "track_name")
#get_all_tracks_with_popularity("99", "*")

In [ ]:
# compare popular tracks to not popular tracks
popular_valence = find_average_of_property_with_popularity_range("valence", 90, 99)
unpopular_valence = find_average_of_property_with_popularity_range("valence", 0, 9)

popular_danceability = find_average_of_property_with_popularity_range("danceability", 90, 99)
unpopular_danceability = find_average_of_property_with_popularity_range("danceability", 0, 9)

popular_energy = find_average_of_property_with_popularity_range("energy", 90, 99)
unpopular_energy = find_average_of_property_with_popularity_range("energy", 0, 9)

popular_liveness = find_average_of_property_with_popularity_range("liveness", 90, 99)
unpopular_liveness = find_average_of_property_with_popularity_range("liveness", 0, 9)

In [ ]:
def print_popular_vs_unpopular(l1,l2):
    assert len(l1) == len(l2)
    s = "popular    VS    unpopular \n"
    for i in range(len(l1)):
        s += str(round(l1[i],2)) + " \t \t "+ str(round(l2[i],2)) + " \n"
    return s


In [ ]:
print(print_popular_vs_unpopular(popular_energy, unpopular_energy))
print(print_popular_vs_unpopular(popular_liveness, unpopular_liveness))
print(print_popular_vs_unpopular(popular_danceability, unpopular_danceability))
print(print_popular_vs_unpopular(popular_valence, unpopular_valence))

### Most popular years in music and their genres

In [ ]:
## create table 3

try:
    session.execute("""
        CREATE TABLE IF NOT EXISTS release_year_album_popularity (
            track_name TEXT,
            track_id TEXT,
            track_number TEXT,
            duration_ms TEXT,
            album_type TEXT,
            artists TEXT,
            total_tracks TEXT,
            album_name TEXT,
            release_date TEXT,
            label TEXT,
            album_popularity INT,
            album_id TEXT,
            artist_id TEXT,
            duration_sec TEXT,
            release_year TEXT,
            track_popularity TEXT,
            release_month TEXT,
            release_day TEXT,
            danceability FLOAT,
            loudness FLOAT,
            energy FLOAT,
            speechiness FLOAT,
            acousticness FLOAT,
            instrumentalness FLOAT,
            liveness FLOAT,
            valence FLOAT,
            tempo FLOAT,
            artist_genres TEXT,
            PRIMARY KEY (release_year, album_popularity));
    """)
except Exception as e:
    print(e)

In [ ]:
# session.execute("DROP TABLE spotify.release_year_album_popularity;")

In [ ]:
# result = session.execute("DESCRIBE TABLE release_year_album_popularity")

In [ ]:
# # Iterate through the rows and print each row
# for row in result:
#     print(row)

In [ ]:
# insert release year datafram to casandra table

types = [str,str,str,str,str,str,str,str,str,str,int,str,str,str,str,str,str,str,float,float,float,float,float,float,float,float,float,str]


statement_track_info = '''
INSERT INTO release_year_album_popularity
(track_name, track_id,track_number ,duration_ms,album_type, artists,total_tracks ,album_name, release_date,label, album_popularity, album_id, artist_id, duration_sec, track_popularity, release_year, release_month, release_day, danceability, loudness, energy, speechiness, acousticness, instrumentalness, liveness, valence,tempo, artist_genres)
 VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?,?);
'''
prepared_query = session.prepare(statement_track_info)

# for max performance we should split out csv by partition
df_to_cassandra(df_sorted_release_year, prepared_query, types)


In [ ]:

result_set = session.execute("""
SELECT release_year, AVG(album_popularity) AS avg_popularity, COUNT(*) AS num_albums
FROM release_year_album_popularity
GROUP BY release_year;""")
rows = list(result_set)


avg_popularity_by_years_df = pd.DataFrame(rows)

In [ ]:
avg_popularity_by_years_df.sort_values(by='avg_popularity', ascending=True, inplace=True)

In [ ]:
avg_popularity_by_years_df = avg_popularity_by_years_df[avg_popularity_by_years_df['num_albums'] > 15]

In [ ]:
avg_popularity_by_years_df

In [ ]:
import matplotlib.pyplot as plt
# Plot the data
plt.scatter(avg_popularity_by_years_df['release_year'], avg_popularity_by_years_df['num_albums'])
plt.title(f'Number of album released during the years')
plt.xlabel('Release Year')
plt.xticks([])
plt.ylabel('num_albums')
plt.show()

In [ ]:
import matplotlib.pyplot as plt
# Plot the data
plt.scatter(avg_popularity_by_years_df['release_year'], avg_popularity_by_years_df['avg_popularity'])
plt.title(f'Popularity of albums during the years')
plt.xlabel('Release Year')
plt.xticks([])
plt.ylabel('avg_popularity')
plt.show()

In [ ]:
max_avg_popularity_row = avg_popularity_by_years_df.loc[avg_popularity_by_years_df['avg_popularity'].idxmax()]

print("release_year with the maximum average popularity:")
print(max_avg_popularity_row['release_year'])

In [ ]:

result_set = session.execute("""
SELECT *
FROM release_year_album_popularity
WHERE release_year='2023';""")
rows = list(result_set)


most_popular_year_in_music_df = pd.DataFrame(rows)

In [ ]:
# Assuming 'album_popularity' is a column in most_popular_year_in_music_df
album_popularity_values = most_popular_year_in_music_df['album_popularity'].astype(float)

# Plotting the distribution as a histogram
plt.hist(album_popularity_values, edgecolor='black')  # Adjust the number of bins as needed
plt.title('Distribution of Album Popularity')
plt.xlabel('Album Popularity')
plt.ylabel('Frequency')
plt.show()

In [ ]:
most_popular_albums_2023 = most_popular_year_in_music_df[most_popular_year_in_music_df['album_popularity'] > 80]

In [ ]:
most_popular_albums_2023

In [ ]:
# pip install wordcloud matplotlib


In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt

all_genres = ' '.join(most_popular_albums_2023['artist_genres'].astype(str))


wordcloud = WordCloud(width=800, height=400, background_color='white').generate(all_genres)


plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.title('Most popular Genres in 2023')
plt.show()


In [ ]:
least_popular_albums_2023 = most_popular_year_in_music_df[most_popular_year_in_music_df['album_popularity'] < 25]

In [ ]:
least_popular_albums_2023

In [ ]:
all_genres = ' '.join(least_popular_albums_2023['artist_genres'].astype(str))


wordcloud = WordCloud(width=800, height=400, background_color='white').generate(all_genres)


plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.title('Least popular Genres in 2023')
plt.show()


In [ ]:

result_set = session.execute("SELECT release_year, AVG(album_popularity) AS avg_popularity, COLLECT(artist_genres) AS genres_list FROM albums_with_features GROUP BY release_year")
rows = list(result_set)


danceability_vs_popularity_df = pd.DataFrame(rows)

In [ ]:
cluster.shutdown()

## <u>non_batch solution for comparisson: <u>

In [ ]:
try:
    session.execute("""
        CREATE TABLE IF NOT EXISTS track_info_naive_2 (
            track_name TEXT,
            track_id TEXT,
            track_number TEXT,
            duration_ms TEXT,
            album_type TEXT,
            artists TEXT,
            total_tracks TEXT,
            album_name TEXT,
            release_date TEXT,
            label TEXT,
            album_popularity TEXT,
            album_id TEXT,
            artist_id TEXT,
            artist_0 TEXT,
            artist_1 TEXT,
            artist_2 TEXT,
            artist_3 TEXT,
            artist_4 TEXT,
            artist_5 TEXT,
            artist_6 TEXT,
            artist_7 TEXT,
            artist_8 TEXT,
            artist_9 TEXT,
            artist_10 TEXT,
            artist_11 TEXT,
            duration_sec TEXT,
            PRIMARY KEY (album_popularity, track_id));
    """)
except Exception as e:
    print(e)

## naive implementation for comparison
def df_to_cassandra_naive(path, prepared_query, types, batch_size=20, has_header=True):
        for index, row in path.iterrows():
            converted_vals = convert_to_types(types, row.tolist())
            session.execute(prepared_query, converted_vals)
            #print("yes")

types = [str]*26

statement = '''
INSERT INTO track_info_naive_2
(track_name,track_id,track_number,duration_ms,album_type,artists,total_tracks,album_name,release_date,label,album_popularity,album_id,artist_id,duration_sec,release_year,track_popularity,release_month,release_day,danceability,loudness,energy,speechiness,acousticness,instrumentalness,liveness,valence,tempo)
VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?);
'''
prepared_query = session.prepare(statement)

# for max performance we should split out csv by partition
df_to_cassandra_naive(df, prepared_query, types)

